# Workspace initialization

In [1]:
from azureml.core import Workspace, Experiment

# Initializing the workspace
workspace_name = "quick-starts-ws-127597"
ws = Workspace.get(name=workspace_name)

# Define an experiment
exp = Experiment(workspace=ws, name=workspace_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127597
Azure region: southcentralus
Subscription id: 888519c8-2387-461a-aff3-b31b86e2438e
Resource group: aml-quickstarts-127597


## Creating a compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster_name = "udacity-project"
# Check if a compute cluster already exists
try:
    print("Trying to connect to an existing cluster...")
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    print("Creating a compute cluster...")
    compute_configuration = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_configuration)
    compute_cluster.wait_for_completion(show_output=True)
print("Success!")

Trying to connect to an existing cluster...
Creating a compute cluster...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Success!


# Hyperparameter tuning with Hyperdrive

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--C": uniform(0.1, 1.0),
    "--max_iter": choice(range(1, 1000))
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('train.py', './training/')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./training",
              entry_script="train.py",
              compute_target="udacity-project")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10,
                                     policy=policy,
                                     max_concurrent_runs=2)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run = exp.submit(hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [5]:
# See the progress of the run

RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
pickle_filename = "outputs/" + best_run.id + ".pkl"
best_run.download_file(pickle_filename)
print("Best model saved.")

Best model saved.


In [7]:
# Print the best parameters
parameters = best_run.get_details()['runDefinition']['arguments']
print('Regularization Strength:',parameters[1])
print('Maximum number of Iterations:',parameters[3])

Regularization Strength: 0.6304654789800282
Maximum number of Iterations: 242


# Azure AutoML

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=data_url)

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
from azureml.train.automl import AutoMLConfig

# Use of X= and y= are deprecated on AutoMLConfig instantiation
# See more at the official docs: https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
training_data = x.join(y)

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=3)

In [11]:
# Submit your automl run

automl_run = exp.submit(config=automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_7dd83831-c436-41dd-ad98-c1348563baae

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [17]:
# Retrieve and save your best automl model.

automl_best_run, automl_model = automl_run.get_output()
joblib.dump(automl_model, "automl-model.pkl")
automl_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  learning_rate='constant',
                                                                                                  loss='modified_huber',
                                                                                                  max_iter=1000,
                                  

In [18]:
# See more details at the Azure Machine Learning Studio
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-127597,AutoML_7dd83831-c436-41dd-ad98-c1348563baae_52,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
# Delete the compute target for cost-saving
compute_cluster.delete()